In [ ]:
!git clone https://github.com/pooja-premnath/CheckThat-Task2-Subjectivity

fatal: destination path 'CheckThat-Task2-Subjectivity' already exists and is not an empty directory.


In [ ]:
import pandas as pd


file_path = "/content/CheckThat-Task2-Subjectivity/dev_test_en.tsv"
df_test = pd.read_csv(file_path, sep='\t')


print(df_test.head())


                            sentence_id  \
0  8745d4da-91c9-4538-acee-b0e7b1c413fd   
1  43de04ad-d0ac-4852-9b4e-cf0bca066188   
2  e00b66ee-720a-47e3-a0fb-0e2445b89af6   
3  0b95d635-f821-45dd-9f33-b05d63629195   
4  5ba3117b-3ef9-4815-acb4-a263d3c816bc   

                                            sentence label  
0  Who will redistribute the hoarded wealth that ...  SUBJ  
1  What we don’t need is the indiscriminate influ...  SUBJ  
2  The Social Distance Between Us shows every sig...   OBJ  
3  History shows that McCarthy and McConnell, lik...   OBJ  
4  So while it’s not hard to reach a banal point ...  SUBJ  


In [ ]:
import pandas as pd


file_path = "/content/CheckThat-Task2-Subjectivity/train_en.tsv"
df_train = pd.read_csv(file_path, sep='\t')


print(df_train.head())

                            sentence_id  \
0  b9e1635a-72aa-467f-86d6-f56ef09f62c3   
1  f99b5143-70d2-494a-a2f5-c68f10d09d0a   
2  4076639c-aa56-4202-ae0f-9d9217f8da68   
3  b057c366-698e-419d-a284-9b16d835c64e   
4  a5a9645e-7850-41ba-90a2-5def725cd5b8   

                                            sentence label  solved_conflict  
0  Gone are the days when they led the world in r...  SUBJ             True  
1  The trend is expected to reverse as soon as ne...   OBJ            False  
2             But there is the specious point again.   OBJ            False  
3  He added he wouldn’t be surprised to see a new...   OBJ            False  
4  Not less government, you see; the same amount ...  SUBJ            False  


In [ ]:
len(df_train)

830

In [ ]:



import pandas as pd


file_path = "/content/CheckThat-Task2-Subjectivity/dev_en.tsv"
df_dev = pd.read_csv(file_path, sep='\t')


print(df_dev.head())

                            sentence_id  \
0  ab677701-ae20-42b4-89f2-ddf1eb71b8b7   
1  56164d11-a7f5-4ac5-8dde-681e6f3436e1   
2  676d7dfd-f9d9-42fd-9b00-2165f4576cd7   
3  f3c8718e-d553-4730-89d5-5077c96de10a   
4  10cb731a-0d62-4e85-b101-a51b48a20219   

                                            sentence label  solved_conflict  
0  (It’s also true that Alien Nation was “a maste...  SUBJ            False  
1  It’s all justified in the name of racial “equi...  SUBJ            False  
2  These issues include punishing Sanctuary Citie...   OBJ            False  
3  Only 20 percent of voters support it, while a ...   OBJ            False  
4  In contrast, in 2017 fewer than half of Republ...   OBJ            False  


# Adding POS Tags, Verb Form Features

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')




def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    pos_tags = pos_tag(tokens)
    return pos_tags


def extract_features(pos_tags):
    features = {}
    pos_tags_dict = dict(pos_tags)


    if '"' in pos_tags_dict.values():  # Quotations
        features['Quotations'] = 1
    if 'VBD' in pos_tags_dict.values():  # Past tense verbs
        features['Past_Tense_Verbs'] = 1
    if 'PRP' in pos_tags_dict.values():  # Third person pronouns
        features['Third_Person_Pronouns'] = 1
    if any(tag.isdigit() or tag == 'CD' for tag in pos_tags_dict.values()):  # Numerical values and dates
        features['Numerical_Values_Dates'] = 1


    if 'VB' in pos_tags_dict.values() or 'MD' in pos_tags_dict.values():  # Imperative verbs and modals
        features['Imperative_Verbs'] = 1
    if any(tag in ['!', '?'] for tag in pos_tags_dict.values()):  # Exclamation and question marks
        features['Exclamation_Question_Marks'] = 1
    if 'PRP' in pos_tags_dict.values() and ('VBZ' in pos_tags_dict.values() or 'VBP' in pos_tags_dict.values()):  # First and second person pronouns with present tense verbs
        features['First_Second_Pronouns_Present_Tense'] = 1
    if 'RB' in pos_tags_dict.values() and ('JJR' in pos_tags_dict.values() or 'JJS' in pos_tags_dict.values()):  # Comparative and superlative adverbs and adjectives
        features['Comparative_Superlative_Adverbs_Adjectives'] = 1
    if ('VBZ' in pos_tags_dict.values() or 'VBP' in pos_tags_dict.values()) and 'PRP' in pos_tags_dict.values():  # Present tense verbs with first and second pronouns
        features['Present_Tense_Verbs_First_Second_Pronouns'] = 1

    return features


df_train['POS_Tags'] = df_train['sentence'].apply(preprocess_text)
df_train['Features'] = df_train['POS_Tags'].apply(extract_features)


features_df = pd.DataFrame(list(df_train['Features']))


df_final = pd.concat([df_train, features_df], axis=1)


print(df_final)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


                              sentence_id  \
0    b9e1635a-72aa-467f-86d6-f56ef09f62c3   
1    f99b5143-70d2-494a-a2f5-c68f10d09d0a   
2    4076639c-aa56-4202-ae0f-9d9217f8da68   
3    b057c366-698e-419d-a284-9b16d835c64e   
4    a5a9645e-7850-41ba-90a2-5def725cd5b8   
..                                    ...   
825  9a0f5eec-cc36-49b8-88eb-20ad2c056eaa   
826  73545884-adf8-480c-a8b5-e65128ba8e91   
827  a8825286-21a4-46c9-9410-c0e7e183d708   
828  c984fc97-2604-4690-a2c0-d748703663af   
829  1b79225e-db68-43ef-a025-544b23a6c058   

                                              sentence label  solved_conflict  \
0    Gone are the days when they led the world in r...  SUBJ             True   
1    The trend is expected to reverse as soon as ne...   OBJ            False   
2               But there is the specious point again.   OBJ            False   
3    He added he wouldn’t be surprised to see a new...   OBJ            False   
4    Not less government, you see; the same amount ...

In [ ]:

df_test['POS_Tags'] = df_test['sentence'].apply(preprocess_text)
df_test['Features'] = df_test['POS_Tags'].apply(extract_features)

features_test_df = pd.DataFrame(list(df_test['Features']))

df_test_final = pd.concat([df_test, features_test_df], axis=1)

print(df_test_final)


                              sentence_id  \
0    8745d4da-91c9-4538-acee-b0e7b1c413fd   
1    43de04ad-d0ac-4852-9b4e-cf0bca066188   
2    e00b66ee-720a-47e3-a0fb-0e2445b89af6   
3    0b95d635-f821-45dd-9f33-b05d63629195   
4    5ba3117b-3ef9-4815-acb4-a263d3c816bc   
..                                    ...   
238  62cff7b0-e459-4cbe-b534-6a814a50302e   
239  dfd1bc2d-5419-46fd-84a1-ac4d2ee2138d   
240  e398e3aa-2af1-45d4-9672-d709d50b6aed   
241  7db63c04-f58e-4b77-93d4-d1e22fb7a376   
242  1f703ae9-0468-4b33-ae3e-fc3368f334a4   

                                              sentence label  \
0    Who will redistribute the hoarded wealth that ...  SUBJ   
1    What we don’t need is the indiscriminate influ...  SUBJ   
2    The Social Distance Between Us shows every sig...   OBJ   
3    History shows that McCarthy and McConnell, lik...   OBJ   
4    So while it’s not hard to reach a banal point ...  SUBJ   
..                                                 ...   ...   
238  The po

In [ ]:
df_final.fillna(0, inplace=True)
df_test_final.fillna(0, inplace=True)




In [ ]:
df_final.drop(columns=['solved_conflict'], inplace=True)

In [ ]:
df_final.head()

,sentence_id,sentence,label,POS_Tags,Features,Past_Tense_Verbs,Imperative_Verbs,Numerical_Values_Dates,Comparative_Superlative_Adverbs_Adjectives,Third_Person_Pronouns,First_Second_Pronouns_Present_Tense,Present_Tense_Verbs_First_Second_Pronouns
0,b9e1635a-72aa-467f-86d6-f56ef09f62c3,Gone are the days when they led the world in r...,SUBJ,"[(gone, VBN), (days, NNS), (led, VBD), (world,...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,f99b5143-70d2-494a-a2f5-c68f10d09d0a,The trend is expected to reverse as soon as ne...,OBJ,"[(trend, NN), (expected, VBD), (reverse, JJ), ...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4076639c-aa56-4202-ae0f-9d9217f8da68,But there is the specious point again.,OBJ,"[(specious, JJ), (point, NN)]",{},0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,b057c366-698e-419d-a284-9b16d835c64e,He added he wouldn’t be surprised to see a new...,OBJ,"[(added, VBD), (surprised, JJ), (see, VB), (ne...","{'Past_Tense_Verbs': 1, 'Imperative_Verbs': 1}",1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,a5a9645e-7850-41ba-90a2-5def725cd5b8,"Not less government, you see; the same amount ...",SUBJ,"[(less, RBR), (government, NN), (see, NN), (am...",{},0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_test_final.head()

,sentence_id,sentence,label,POS_Tags,Features,Past_Tense_Verbs,Third_Person_Pronouns,First_Second_Pronouns_Present_Tense,Present_Tense_Verbs_First_Second_Pronouns,Imperative_Verbs,Numerical_Values_Dates,Comparative_Superlative_Adverbs_Adjectives
0,8745d4da-91c9-4538-acee-b0e7b1c413fd,Who will redistribute the hoarded wealth that ...,SUBJ,"[(redistribute, NN), (hoarded, VBD), (wealth, ...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,43de04ad-d0ac-4852-9b4e-cf0bca066188,What we don’t need is the indiscriminate influ...,SUBJ,"[(need, NN), (indiscriminate, NN), (influxes, ...",{},0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,e00b66ee-720a-47e3-a0fb-0e2445b89af6,The Social Distance Between Us shows every sig...,OBJ,"[(social, JJ), (distance, NN), (us, PRP), (sho...","{'Third_Person_Pronouns': 1, 'First_Second_Pro...",0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,0b95d635-f821-45dd-9f33-b05d63629195,"History shows that McCarthy and McConnell, lik...",OBJ,"[(history, NN), (shows, NNS), (mccarthy, VBP),...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5ba3117b-3ef9-4815-acb4-a263d3c816bc,So while it’s not hard to reach a banal point ...,SUBJ,"[(hard, JJ), (reach, VBP), (banal, JJ), (point...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Get the column order from df_final
column_order = df_final.columns.tolist()

# Reorder columns in df_test_final to match df_final
df_test_final = df_test_final[column_order]



In [ ]:
df_final.columns

Index(['sentence_id', 'sentence', 'label', 'POS_Tags', 'Features',
       'Past_Tense_Verbs', 'Imperative_Verbs', 'Numerical_Values_Dates',
       'Comparative_Superlative_Adverbs_Adjectives', 'Third_Person_Pronouns',
       'First_Second_Pronouns_Present_Tense',
       'Present_Tense_Verbs_First_Second_Pronouns'],
      dtype='object')

In [ ]:
df_test_final.columns

Index(['sentence_id', 'sentence', 'label', 'POS_Tags', 'Features',
       'Past_Tense_Verbs', 'Imperative_Verbs', 'Numerical_Values_Dates',
       'Comparative_Superlative_Adverbs_Adjectives', 'Third_Person_Pronouns',
       'First_Second_Pronouns_Present_Tense',
       'Present_Tense_Verbs_First_Second_Pronouns'],
      dtype='object')

In [ ]:
pip install spacy scikit-learn


In [ ]:
import spacy
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
nlp = spacy.load("en_core_web_sm")

def spacy_vectorize(text):
    doc = nlp(text)
    return doc.vector


In [ ]:
df_final['Sentence_Vectors'] = df_final['sentence'].apply(spacy_vectorize)
df_test_final['Sentence_Vectors'] = df_test_final['sentence'].apply(spacy_vectorize)


In [ ]:
df_final.head()

,sentence_id,sentence,label,POS_Tags,Features,Past_Tense_Verbs,Imperative_Verbs,Numerical_Values_Dates,Comparative_Superlative_Adverbs_Adjectives,Third_Person_Pronouns,First_Second_Pronouns_Present_Tense,Present_Tense_Verbs_First_Second_Pronouns,Sentence_Vectors
0,b9e1635a-72aa-467f-86d6-f56ef09f62c3,Gone are the days when they led the world in r...,SUBJ,"[(gone, VBN), (days, NNS), (led, VBD), (world,...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.04463423, -0.04594215, 0.181084, -0.0955818..."
1,f99b5143-70d2-494a-a2f5-c68f10d09d0a,The trend is expected to reverse as soon as ne...,OBJ,"[(trend, NN), (expected, VBD), (reverse, JJ), ...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.23961264, -0.19176887, 0.107891046, -0.2269..."
2,4076639c-aa56-4202-ae0f-9d9217f8da68,But there is the specious point again.,OBJ,"[(specious, JJ), (point, NN)]",{},0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.28663486, -0.47321332, 0.23614675, -0.25727..."
3,b057c366-698e-419d-a284-9b16d835c64e,He added he wouldn’t be surprised to see a new...,OBJ,"[(added, VBD), (surprised, JJ), (see, VB), (ne...","{'Past_Tense_Verbs': 1, 'Imperative_Verbs': 1}",1.0,1.0,0.0,0.0,0.0,0.0,0.0,"[-0.07255475, -0.45236158, 0.0482783, -0.15360..."
4,a5a9645e-7850-41ba-90a2-5def725cd5b8,"Not less government, you see; the same amount ...",SUBJ,"[(less, RBR), (government, NN), (see, NN), (am...",{},0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.2134672, -0.21399115, -0.031350687, -0.0739..."


In [ ]:
df_test_final.head()

,sentence_id,sentence,label,POS_Tags,Features,Past_Tense_Verbs,Imperative_Verbs,Numerical_Values_Dates,Comparative_Superlative_Adverbs_Adjectives,Third_Person_Pronouns,First_Second_Pronouns_Present_Tense,Present_Tense_Verbs_First_Second_Pronouns,Sentence_Vectors
0,8745d4da-91c9-4538-acee-b0e7b1c413fd,Who will redistribute the hoarded wealth that ...,SUBJ,"[(redistribute, NN), (hoarded, VBD), (wealth, ...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1189769, -0.251255, 0.15438071, -0.1836857,..."
1,43de04ad-d0ac-4852-9b4e-cf0bca066188,What we don’t need is the indiscriminate influ...,SUBJ,"[(need, NN), (indiscriminate, NN), (influxes, ...",{},0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.03890463, -0.03602256, -0.076018006, 0.188..."
2,e00b66ee-720a-47e3-a0fb-0e2445b89af6,The Social Distance Between Us shows every sig...,OBJ,"[(social, JJ), (distance, NN), (us, PRP), (sho...","{'Third_Person_Pronouns': 1, 'First_Second_Pro...",0.0,0.0,0.0,0.0,1.0,1.0,1.0,"[0.14624265, -0.22532664, -0.08710448, 0.34260..."
3,0b95d635-f821-45dd-9f33-b05d63629195,"History shows that McCarthy and McConnell, lik...",OBJ,"[(history, NN), (shows, NNS), (mccarthy, VBP),...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.018699985, -0.26669225, -0.085256174, 0.23..."
4,5ba3117b-3ef9-4815-acb4-a263d3c816bc,So while it’s not hard to reach a banal point ...,SUBJ,"[(hard, JJ), (reach, VBP), (banal, JJ), (point...",{'Past_Tense_Verbs': 1},1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.16485395, -0.31768924, 0.064073764, 0.09066..."


In [ ]:
label_encoder = LabelEncoder()
df_final['Encoded_Label'] = label_encoder.fit_transform(df_final['label'])
df_test_final['Encoded_Label'] = label_encoder.transform(df_test_final['label'])


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming you have already created the label_encoder object and encoded your data
label_encoder = LabelEncoder()
df_final['Encoded_Label'] = label_encoder.fit_transform(df_final['label'])
df_test_final['Encoded_Label'] = label_encoder.transform(df_test_final['label'])

# Get the original classes from the label_encoder object
original_classes = label_encoder.classes_

# Print the encoded labels along with their corresponding original classes
for encoded_label, original_label in zip(df_final['Encoded_Label'], df_final['label']):
    print(f"Encoded Label: {encoded_label}, Original Label: {original_classes[encoded_label]}")


#1= SUBJ, 0=OBJ

Encoded Label: 1, Original Label: SUBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 1, Original Label: SUBJ
Encoded Label: 1, Original Label: SUBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 1, Original Label: SUBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 1, Original Label: SUBJ
Encoded Label: 1, Original Label: SUBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 1, Original Label: SUBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 1, Original Label: SUBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Encoded Label: 0, Original Label: OBJ
Enco

In [ ]:
df_final.drop(columns=['sentence'], inplace=True)
df_test_final.drop(columns=['sentence'], inplace=True)

In [ ]:
df_final.drop(columns=['label'], inplace=True)
df_test_final.drop(columns=['label'], inplace=True)

In [ ]:
df_final.drop(columns=['POS_Tags', "Features"], inplace=True)
df_test_final.drop(columns=['POS_Tags', "Features"], inplace=True)

In [ ]:
df_final.head()

,sentence_id,Past_Tense_Verbs,Imperative_Verbs,Numerical_Values_Dates,Comparative_Superlative_Adverbs_Adjectives,Third_Person_Pronouns,First_Second_Pronouns_Present_Tense,Present_Tense_Verbs_First_Second_Pronouns,Sentence_Vectors,Encoded_Label
0,b9e1635a-72aa-467f-86d6-f56ef09f62c3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.04463423, -0.04594215, 0.181084, -0.0955818...",1
1,f99b5143-70d2-494a-a2f5-c68f10d09d0a,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.23961264, -0.19176887, 0.107891046, -0.2269...",0
2,4076639c-aa56-4202-ae0f-9d9217f8da68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.28663486, -0.47321332, 0.23614675, -0.25727...",0
3,b057c366-698e-419d-a284-9b16d835c64e,1.0,1.0,0.0,0.0,0.0,0.0,0.0,"[-0.07255475, -0.45236158, 0.0482783, -0.15360...",0
4,a5a9645e-7850-41ba-90a2-5def725cd5b8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.2134672, -0.21399115, -0.031350687, -0.0739...",1


In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score


X_train_vectors = np.array(df_final['Sentence_Vectors'].tolist())
X_test_vectors = np.array(df_test_final['Sentence_Vectors'].tolist())


X_train = df_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)  # Exclude 'Sentence_id' column
X_train = np.hstack((X_train, X_train_vectors))  # Concatenate the sentence vectors

y_train = df_final['Encoded_Label']


X_test = df_test_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)  # Exclude 'Sentence_id' column
X_test = np.hstack((X_test, X_test_vectors))

y_test = df_test_final['Encoded_Label']


svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)


y_pred = svm_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Macro-Averaged F1 Score:", macro_f1)


Accuracy: 0.5637860082304527
Macro-Averaged F1 Score: 0.5402327573896901


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Prepare the data
X_train_vectors = np.array(df_final['Sentence_Vectors'].tolist())
X_test_vectors = np.array(df_test_final['Sentence_Vectors'].tolist())

X_train = df_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)
X_train = np.hstack((X_train, X_train_vectors))

y_train = df_final['Encoded_Label']

X_test = df_test_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)
X_test = np.hstack((X_test, X_test_vectors))

y_test = df_test_final['Encoded_Label']

# Initialize classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'SVM': SVC(kernel='linear', random_state=42)
}

# Train and evaluate classifiers
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    results[name] = {'Accuracy': accuracy, 'Macro-F1': macro_f1}

# Print results
for name, metrics in results.items():
    print(f"{name}: Accuracy = {metrics['Accuracy']}, Macro-F1 = {metrics['Macro-F1']}")


Logistic Regression: Accuracy = 0.5679012345679012, Macro-F1 = 0.5378136038402319
Random Forest: Accuracy = 0.5349794238683128, Macro-F1 = 0.45717109815162593
Gradient Boosting: Accuracy = 0.551440329218107, Macro-F1 = 0.5082706766917293
K-Nearest Neighbors: Accuracy = 0.6008230452674898, Macro-F1 = 0.6001458888191487
SVM: Accuracy = 0.5637860082304527, Macro-F1 = 0.5402327573896901


In [ ]:
import pandas as pd


file_path = "/content/CheckThat-Task2-Subjectivity/test_en.tsv"
df_submission = pd.read_csv(file_path, sep='\t')


print(df_submission.head())

                            sentence_id  \
0  44f33601-157a-42ce-aa9f-0f7d305501f2   
1  6f9e0f53-f76c-432f-bbea-b78400d600b8   
2  61f93bdc-4c3e-4963-926c-0bbf139b44b9   
3  902148ec-dda3-4736-b318-0f20c63a1cf3   
4  065b1996-4b40-4c74-9f62-afb44f69834e   

                                            sentence  
0  Blanco established himself earlier in his care...  
1  RULE 13: ARTIFICIAL INTELLIGENCE  Not only thi...  
2  The valuation is required by law and the figur...  
3  A sip can really hit the spot after a long bik...  
4                                          Lobster!"  


In [ ]:
df_submission['POS_Tags'] = df_submission['sentence'].apply(preprocess_text)
df_submission['Features'] = df_submission['POS_Tags'].apply(extract_features)

features_test_df = pd.DataFrame(list(df_submission['Features']))

df_submission_final = pd.concat([df_submission, features_test_df], axis=1)




In [ ]:
df_submission_final['Sentence_Vectors'] = df_submission_final['sentence'].apply(spacy_vectorize)


In [ ]:

column_order = df_final.drop('Encoded_Label', axis=1).columns.tolist()

df_submission_final = df_submission_final[column_order]




In [ ]:
df_submission_final.fillna(0,inplace=True)

<ipython-input-74-f0582ae5ff49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submission_final.fillna(0,inplace=True)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Prepare the data
X_train_vectors = np.array(df_final['Sentence_Vectors'].tolist())
X_test_vectors = np.array(df_test_final['Sentence_Vectors'].tolist())
X_submission_vectors = np.array(df_submission_final['Sentence_Vectors'].tolist())

X_train = df_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)
X_train = np.hstack((X_train, X_train_vectors))
X_test = df_test_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)
X_test = np.hstack((X_test, X_test_vectors))
X_submission = df_submission_final.drop(['sentence_id', 'Sentence_Vectors'], axis=1)
X_submission = np.hstack((X_submission, X_submission_vectors))

y_train = df_final['Encoded_Label']
y_test = df_test_final['Encoded_Label']

# Initialize classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'SVM': SVC(kernel='linear', random_state=42)
}

# Train, evaluate, predict, and write predicted labels to TSV file for each classifier
results = {}
for name, clf in classifiers.items():
    # Train and evaluate classifiers
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    results[name] = {'Accuracy': accuracy, 'Macro-F1': macro_f1}

    # Predict labels for df_final_submission
    y_submission_pred = clf.predict(X_submission)

    # Create a DataFrame with sentence_id and predicted labels
    predicted_data = pd.DataFrame({'sentence_id': df_submission_final['sentence_id'], 'Predicted_Label': y_submission_pred})

    # Save the predicted data to a TSV file
    predicted_data.to_csv(f'{name}_predicted_labels.tsv', sep='\t', index=False)

# Print results
for name, metrics in results.items():
    print(f"{name}: Accuracy = {metrics['Accuracy']}, Macro-F1 = {metrics['Macro-F1']}")


Logistic Regression: Accuracy = 0.5679012345679012, Macro-F1 = 0.5378136038402319
Random Forest: Accuracy = 0.5349794238683128, Macro-F1 = 0.45717109815162593
Gradient Boosting: Accuracy = 0.551440329218107, Macro-F1 = 0.5082706766917293
K-Nearest Neighbors: Accuracy = 0.6008230452674898, Macro-F1 = 0.6001458888191487
SVM: Accuracy = 0.5637860082304527, Macro-F1 = 0.5402327573896901


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Prepare the data
X_train_vectors = np.array(df_final['Sentence_Vectors'].tolist())
X_test_vectors = np.array(df_test_final['Sentence_Vectors'].tolist())
X_submission_vectors = np.array(df_submission_final['Sentence_Vectors'].tolist())

X_train = df_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)
X_train = np.hstack((X_train, X_train_vectors))
X_test = df_test_final.drop(['sentence_id', 'Sentence_Vectors', 'Encoded_Label'], axis=1)
X_test = np.hstack((X_test, X_test_vectors))
X_submission = df_submission_final.drop(['sentence_id', 'Sentence_Vectors'], axis=1)
X_submission = np.hstack((X_submission, X_submission_vectors))

y_train = df_final['Encoded_Label']
y_test = df_test_final['Encoded_Label']

# Initialize classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'SVM': SVC(kernel='linear', random_state=42)
}

# Train, evaluate, predict, and write predicted labels to TSV file for each classifier
results = {}
for name, clf in classifiers.items():
    # Train and evaluate classifiers
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    results[name] = {'Accuracy': accuracy, 'Macro-F1': macro_f1}

    # Predict labels for df_final_submission
    y_submission_pred = clf.predict(X_submission)

    # Map predicted labels to 'OBJ' and 'SUBJ'
    y_submission_mapped = ['OBJ' if label == 0 else 'SUBJ' for label in y_submission_pred]

    # Create a DataFrame with sentence_id and predicted labels
    predicted_data = pd.DataFrame({'sentence_id': df_submission_final['sentence_id'], 'label': y_submission_mapped})

    # Save the predicted data to a TSV file
    predicted_data.to_csv(f'{name}_predicted_labels.tsv', sep='\t', index=False)

# Print results
for name, metrics in results.items():
    print(f"{name}: Accuracy = {metrics['Accuracy']}, Macro-F1 = {metrics['Macro-F1']}")


Logistic Regression: Accuracy = 0.5679012345679012, Macro-F1 = 0.5378136038402319
Random Forest: Accuracy = 0.5349794238683128, Macro-F1 = 0.45717109815162593
Gradient Boosting: Accuracy = 0.551440329218107, Macro-F1 = 0.5082706766917293
K-Nearest Neighbors: Accuracy = 0.6008230452674898, Macro-F1 = 0.6001458888191487
SVM: Accuracy = 0.5637860082304527, Macro-F1 = 0.5402327573896901
